In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from os.path import exists
import time

import pandas as pd
from sklearn.model_selection import train_test_split

from data.data_generation import create_artificial_data, load_beans_data
from models.traditional_models import GaussianNBModel, SVM, RandomForest, AdaBoost
from models.myMLP import MyMLP
from conformal_methods.mapie_evaluation import evaluate_mapie
from conformal_methods.byhand_evaluation import score_methode, adaptive_prediction_methode
from conformal_methods.evaluation import evaluate_conformal, epoch_time

from models.skorchMLP import skorchMLP

import torch

from mapie.classification import MapieClassifier
from mapie.metrics import classification_coverage_score, classification_mean_width_score

# Create Data

In [2]:
#dataset="beans"
dataset="artifical"
if dataset=="beans":
    data, classes = load_beans_data()
    n_classes = len(classes.classes_)
    n_features = data.shape[1]-1
else:
    n_classes = 5
    n_features = 20
    classes = None
    data = create_artificial_data(n_features, n_classes)

print('Shape of the data:', data.shape)
print("Classes: ", n_classes, " Features : ", n_features)

#data.head()
#fig = plt.figure(figsize=(8, 6))
#plt.matshow( data.corr(), fignum=fig.number )
#cb = plt.colorbar()
#cb.ax.tick_params(labelsize=14)
#plt.title('Correlation Matrix', fontsize=16);

Shape of the data: (10000, 21)
Classes:  5  Features :  20


In [3]:
Y = data["Class"]
X = data.drop("Class", axis=1)
X_train, X_rest1, y_train, y_rest1 = train_test_split(X, Y, train_size=7000, random_state=2)
X_test, X_rest2, y_test, y_rest2 = train_test_split(X_rest1, y_rest1, train_size=1000, random_state=42)
X_calib, X_new, y_calib, y_new = train_test_split(X_rest2, y_rest2, train_size=1000, random_state=42)

print("Data sizes train: %i, test: %i, calibration: %i, new: %i" % (len(X_train), len(X_test), len(X_calib), len(X_new)))

Data sizes train: 7000, test: 1000, calibration: 1000, new: 1000


# Load and Train models

In [10]:
gaussianNB_model = GaussianNBModel()
gaussianNB_model.fit(X_train, y_train)
gaussianNB_model.predict_acc(X_test, y_test)
print("Gauss acc: ", gaussianNB_model.acc)

svm_model = SVM()
svm_model.fit(X_train, y_train)
svm_model.predict_acc(X_test, y_test)
print("SVM acc: " , svm_model.acc)

#random_forest_model = RandomForest()
#random_forest_model.fit(X_train, y_train)
#random_forest_model.predict(X_test, y_test)
#print("Forest: " , random_forest_model.acc)

#adaBoost_model = AdaBoost()
#adaBoost_model.fit(X_train, y_train)
#adaBoost_model.predict(X_test, y_test)
#print("Ada acc: ", adaBoost_model.acc)

#mlp_model = MyMLP(n_classes, n_features, dataset="beans")
#mlp_model.fitNN(X_train, y_train, X_test, y_test)
#mlp_model.predict_acc(X_test, y_test) 
#print("MLP acc: ", mlp_model.acc)

Gauss acc:  0.777
SVM acc:  0.979


# Conformal prediction using naive score approach

### MAPIE

In [22]:
evaluate_mapie(svm_model.model, X_calib, y_calib, X_new, y_new, alpha=0.1, cv="prefit", method="lac", include_last_label=True,  classes=classes)

Coverage: 89.40%
Avg. set size: 0.899000

   class  coverage  avg. set size
0      0  0.900943       0.905660
1      1  0.930233       0.934884
2      2  0.862222       0.862222
3      3  0.876404       0.882022
4      4  0.900000       0.911765


### By hand

In [7]:
prediction_set = score_methode(svm_model.model, X_calib, y_calib, X_new, alpha=0.01)
evaluate_conformal(prediction_set, y_new, list(set(y_calib)))

Coverage: 98.74%
Avg. set size: 1.283453

   class  coverage  avg. set size
0      0  0.980132       1.264901
1      1  1.000000       1.028409
2      2  0.990530       1.297348
3      3  0.993115       1.294320
4      4  0.980655       1.160714
5      5  0.987143       1.177143
6      6  0.984783       1.490217


In [ ]:
prediction_set = score_methode(gaussianNB_model.model, X_calib, y_calib, X_new, alpha=0.01)
evaluate_conformal(prediction_set, y_new, list(set(y_calib)))

In [ ]:
#prediction_set = score_methode(mlp_model, X_calib, y_calib, X_new, alpha=0.01)
#evaluate_conformal(prediction_set, y_new, list(set(y_calib)))

# Adaptive prediction set

In [8]:
evaluate_mapie(gaussianNB_model.model, X_calib, y_calib, X_new, y_new, alpha=0.05, cv="prefit", method="cumulated_score", include_last_label="randomized", classes=classes)

Coverage: 94.32%
Avg. set size: 1.389720

      class  coverage  avg. set size
0  BARBUNYA  0.894040       1.306843
1    BOMBAY  1.000000       1.994318
2      CALI  0.946970       1.181818
3  DERMASON  0.934596       1.460413
4     HOROZ  0.968750       1.578869
5     SEKER  0.954286       1.451429
6      SIRA  0.938043       1.159783


In [9]:
prediction_set = adaptive_prediction_methode(gaussianNB_model.model, X_calib, y_calib, X_new, alpha=0.01)
evaluate_conformal(prediction_set, y_new, list(set(y_calib)))

Coverage: 99.35%
Avg. set size: 1.723054

   class  coverage  avg. set size
0      0  0.995585       1.878587
1      1  1.000000       1.005682
2      2  0.994318       2.204545
3      3  0.998279       1.670396
4      4  0.986607       1.385417
5      5  0.987143       1.394286
6      6  0.994565       2.070652


In [ ]:
#full conformal prediction
# cave the test set is for evaluating the machine learning step
# new datapoints for evaluating the conformal methode

# see how indexing works
#idx = X_new.index[12]
#print(X_new.loc[[idx]])
#print(y_new.loc[[idx]])

idx = X_new.index[0]
X_full_test = pd.concat([X_train, X_calib, X_new.loc[[idx]]])
y_correct = y_new.loc[[idx]]
y_full_test = pd.concat([y_train, y_test])
y_guess = 1
y_full_test.loc[idx] = y_guess
print(X_new.loc[[idx]])
print(y_correct)
print(y_full_test.loc[[idx]])

In [10]:
# iterate over all test points, idx is the pd index of the test point

def full_conformal(model, X_train, y_train, X_calib, y_calib, X_new, y_new, alpha=0.1):   
    coverage = 0
    prediction_set_size = 0
    y_pred = []
    
    start_time = time.monotonic()
    
    for idx in X_new.index:
        # construct new dataset with test point
        X_full_test = pd.concat([X_train, X_calib, X_new.loc[[idx]]])
        n = len(X_full_test)
        
        # this is the correct label
        y_correct = y_new.loc[[idx]]
        
        prediction_set = []
        
        # iterate over all possible labels in Y
        for y_guess in range(0,n_classes):
            # create new label set
            y_full_test = pd.concat([y_train, y_calib])
            y_full_test.loc[idx] = y_guess
            
            # train model
            model.fit(X_full_test, y_full_test)
            
            # calculate conformal scores s_i on full set
            raw_score_cal = model.model.predict_proba(X_full_test)
            cal_scores = 1 - raw_score_cal[np.arange(n), y_full_test]
            
            # calculate qhat_y as quantile of the scores
            q_level = np.ceil((n + 1) * (1 - alpha)) / n
            qhat = np.quantile(cal_scores, q_level, method='higher')
            
            # check if s_test <= qhat_y
            s_test_raw = model.model.predict_proba(X_full_test.loc[[idx]])
            s_test = 1 - s_test_raw[0, y_guess]
            if s_test <= qhat:
                prediction_set.append(y_guess)
        
        y_pred.append(prediction_set)
        # see if y_correct is in the prediction set
        if y_correct.iloc[0] in prediction_set:
            coverage += 1
        prediction_set_size += len(prediction_set)   
        
    coverage = coverage / len(X_new)  
    prediction_set_size = prediction_set_size / len(X_new)
    
    end_time = time.monotonic()
    return y_pred, coverage, prediction_set_size, epoch_time(start_time, end_time)

In [11]:
y_pred, coverage, prediction_set_size, time = full_conformal(GaussianNBModel(), 
                                                       X_train, y_train, X_calib, y_calib, X_new, y_new,
                                                       alpha=0.1)

In [14]:
prediction_set_size

1.013229234439384